### Import necessary libraries

In [209]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import geopandas as gpd

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
import json    

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

import math



print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


embed_map() function to visualise maps

In [156]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

### Define foursquare Credentials and Version

In [87]:
CLIENT_ID = 'IZ3OANXNNHAGM2NO1FBTZ4WWGRLJPQORTA1EC4LEY3GTWJGO' # your Foursquare ID
CLIENT_SECRET = 'A3DOWBNWOGPZKSVYP04IY2UE1FSFBQNIDSKGDLRYG5XVQAD1' # your Foursquare Secret
VERSION = '20200604'
LIMIT = 100
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: IZ3OANXNNHAGM2NO1FBTZ4WWGRLJPQORTA1EC4LEY3GTWJGO
CLIENT_SECRET:A3DOWBNWOGPZKSVYP04IY2UE1FSFBQNIDSKGDLRYG5XVQAD1


I've said I want to base my new Jazz club in Manhatten, so we will centre our searches on the middle of Manhatten

In [88]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7896239 -73.9598939


Search specifically for the existing jazz clubs

In [152]:
search_query = 'Jazz Club'
# removed radius as there aren't any Jazz clubs in centre of Manhattan!
# radius = 500
print(search_query + ' .... OK!')

Jazz Club .... OK!


define and submit the url...

In [153]:
# url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT
    )

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ede63ef95feaf001b33fb96'},
 'response': {'venues': [{'id': '54ed2952498ec36f92b69d0c',
    'name': "Dizzy's Jazz Club",
    'location': {'address': '10 Columbus Cir',
     'lat': 40.768764,
     'lng': -73.982944,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.768764,
       'lng': -73.982944}],
     'distance': 3027,
     'postalCode': '10019',
     'cc': 'US',
     'neighborhood': "Hell's Kitchen",
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['10 Columbus Cir',
      'New York, NY 10019',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e7931735',
      'name': 'Jazz Club',
      'pluralName': 'Jazz Clubs',
      'shortName': 'Jazz Club',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_jazzclub_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591633256',
    'hasPerk': False},
   {'

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [154]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [157]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# # tranform venues into a dataframe
  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
# filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
# filtered_columns = ['name', 'categories', 'lat', 'lng']
# nearby_venues = nearby_venues.loc[:, filtered_columns]

# # filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# # clean columns and keep everything after the full spot
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(5)

<ipython-input-157-38a06c48ce5a>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,id,name,categories,referralId,hasPerk,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,54ed2952498ec36f92b69d0c,Dizzy's Jazz Club,Jazz Club,v-1591633256,False,10 Columbus Cir,40.768764,-73.982944,"[{'label': 'display', 'lat': 40.768764, 'lng':...",3027,10019,US,Hell's Kitchen,New York,NY,United States,"[10 Columbus Cir, New York, NY 10019, United S...",NaN,NaN
1,4aca81a1f964a52028c220e3,Cecil's Jazz Club & Restaurant,Jazz Club,v-1591633256,False,364 Valley Rd,40.774516,-74.239879,"[{'label': 'display', 'lat': 40.77451572910885...",23659,07052,US,NaN,West Orange,NJ,United States,"[364 Valley Rd, West Orange, NJ 07052, United ...",NaN,NaN
2,52195a0211d247f3c3341b9d,Scat Jazz Club,Lounge,v-1591633256,False,NaN,40.812489,-73.941215,"[{'label': 'display', 'lat': 40.81248873725435...",2992,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",NaN,NaN
3,55a5818d498ef469b908d8e2,Cassandra's Jazz Club,Lounge,v-1591633256,False,NaN,40.813977,-73.944630,"[{'label': 'display', 'lat': 40.81397710077426...",3000,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",NaN,NaN
4,4ecca0d7c2ee2da025cee7ac,The Wood Jazz Club,Jazz Club,v-1591633256,False,Netherwood Cir,40.592676,-74.352658,"[{'label': 'display', 'lat': 40.59267589003585...",39745,08820,US,NaN,Edison,NJ,United States,"[Netherwood Cir, Edison, NJ 08820, United States]",NaN,NaN


Let's take a look at the jazz venues just found

In [306]:
# Create a base map
m_1 = folium.Map(location=[latitude,longitude], zoom_start=11)

# Your code here: Add a marker for each jazz club
for idx, row in nearby_venues[nearby_venues['city']=='New York'].iterrows():
    Marker([row['lat'], row['lng']]).add_to(m_1)


# Show the map
embed_map(m_1, 'jazz.html')

get the new york shapefile

In [297]:
NY_boundaries = gpd.read_file("/Users/mark/Downloads/nynta_20b/nynta.shp")
NY_boundaries.to_crs(epsg=4326, inplace=True)
NY_boundaries.to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
# NY_boundaries.crs=('epsg:4326')
NY_boundaries = NY_boundaries[NY_boundaries['BoroName']=='Manhattan'].head()

In [299]:
jazz_gdf = gpd.GeoDataFrame(nearby_venues, geometry=gpd.points_from_xy(nearby_venues.lng, nearby_venues.lat))
jazz_gdf.crs = ('epsg:4326')
jazz_gdf.to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
print (NY_boundaries.crs)
print (jazz_gdf.crs)

epsg:4326
epsg:4326


In [300]:
jazz_gdf.head()

,id,name,categories,referralId,hasPerk,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id,geometry
0,54ed2952498ec36f92b69d0c,Dizzy's Jazz Club,Jazz Club,v-1591633256,False,10 Columbus Cir,40.768764,-73.982944,"[{'label': 'display', 'lat': 40.768764, 'lng':...",3027,10019,US,Hell's Kitchen,New York,NY,United States,"[10 Columbus Cir, New York, NY 10019, United S...",NaN,NaN,POINT (-73.98294 40.76876)
1,4aca81a1f964a52028c220e3,Cecil's Jazz Club & Restaurant,Jazz Club,v-1591633256,False,364 Valley Rd,40.774516,-74.239879,"[{'label': 'display', 'lat': 40.77451572910885...",23659,07052,US,NaN,West Orange,NJ,United States,"[364 Valley Rd, West Orange, NJ 07052, United ...",NaN,NaN,POINT (-74.23988 40.77452)
2,52195a0211d247f3c3341b9d,Scat Jazz Club,Lounge,v-1591633256,False,NaN,40.812489,-73.941215,"[{'label': 'display', 'lat': 40.81248873725435...",2992,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",NaN,NaN,POINT (-73.94121 40.81249)
3,55a5818d498ef469b908d8e2,Cassandra's Jazz Club,Lounge,v-1591633256,False,NaN,40.813977,-73.944630,"[{'label': 'display', 'lat': 40.81397710077426...",3000,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",NaN,NaN,POINT (-73.94463 40.81398)
4,4ecca0d7c2ee2da025cee7ac,The Wood Jazz Club,Jazz Club,v-1591633256,False,Netherwood Cir,40.592676,-74.352658,"[{'label': 'display', 'lat': 40.59267589003585...",39745,08820,US,NaN,Edison,NJ,United States,"[Netherwood Cir, Edison, NJ 08820, United States]",NaN,NaN,POINT (-74.35266 40.59268)


In [309]:
club = jazz_gdf.iloc[0]
# club
distances = NY_boundaries.geometry.distance(club.geometry)
distances
# print('Closest monitoring station ({} metres):'.format(distances.min()))
# print(jazz_gdf.iloc[distances.idxmax()][["address", "lat", "lng"]])
# NY_boundaries.head()

42    0.001039
64    0.057800
65    0.000000
66    0.001317
67    0.010895
dtype: float64

put the clubs in the neighborhoods

In [305]:
# Create a base map
m_6 = folium.Map(location=[latitude,longitude], zoom_start=10)

mc = MarkerCluster()

# add markers to map
for idx, row in jazz_gdf.iterrows():
    if not math.isnan(row['lng']) and not math.isnan(row['lat']):
#         Marker([row['Latitude'], row['Longitude']]).add_to(m_6)
        mc.add_child(folium.Marker([row['lat'], row['lng']]))
    
# # add markers to map
# for lat, lng, borough, label in zip(manhattan['Latitude'], manhattan['Longitude'], manhattan['Borough'], manhattan['Neighborhood']):
#     label = folium.Popup(label, parse_html=True)
#     folium.Marker(
#         row['lat'], row['lng'],
#         radius=5,
#         popup=row['label'],
#         color='blue',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=0.7,
#         parse_html=False).add_to(map_manhattan)  
    

m_6.add_child(mc)
    
# Uncomment to see a hint
#q_6.hint()

# Show the map
embed_map(m_6, 'q_6.html')

Download the newyork dataset

In [63]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [274]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [275]:
neighborhoods_data = newyork_data['features']

Transform the data into a pandas dataframe

In [276]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


populate the currently empty dataframe

In [277]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

I only want the Manhattan neighborhoods, so reduce neigborhoods down to just manhattan borough

In [310]:
manhattan = neighborhoods.loc[neighborhoods['Borough']=='Manhattan']


In [311]:
# neighborhoods.head()
# nearby_venues.head()

#### Create a map of New York with Manhattan neighborhoods superimposed on top.

In [312]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, label in zip(manhattan['Latitude'], manhattan['Longitude'], manhattan['Borough'], manhattan['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
# add jazz clubs to the same map
# id 	name 	categories 	referralId 	hasPerk 	address 	crossStreet 	lat 	lng 	
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
# HeatMap(data=nearby_venues[['lat', 'lng']], radius=15).add_to(map_manhattan)


map_manhattan

explore Neighborhoods of Manhattan

In [313]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [315]:
venues = getNearbyVenues(names=manhattan['Neighborhood'],
                                   latitudes=manhattan['Latitude'],
                                   longitudes=manhattan['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


which neighborhoods have the current jazz clubs

In [338]:
# print(venues.shape)
venues[venues['Venue Category']=='Jazz Club']['Neighborhood']

382        Central Harlem
475       Upper East Side
1342    Greenwich Village
1502         East Village
1885         West Village
1899         West Village
1940         West Village
Name: Neighborhood, dtype: object

In [341]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,63,63,63,63,63,63
Carnegie Hill,84,84,84,84,84,84
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


how many unique categories

In [344]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 333 uniques categories.


In [347]:
# one hot encoding
manhattan_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.sample(10)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
1634,Tribeca,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2286,Carnegie Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1595,Tribeca,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222,Murray Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3076,Hudson Yards,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1979,Manhattan Valley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2481,Civic Center,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,Upper East Side,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2869,Turtle Bay,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
487,Upper East Side,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [348]:
manhattan_onehot.shape

(3122, 334)

In [349]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.00,0.00,0.000000,0.015873,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.015873,0.00000,0.000000,0.000000
1,Carnegie Hill,0.000000,0.00,0.00,0.000000,0.011905,0.00,0.000000,0.000000,0.011905,...,0.00,0.023810,0.000000,0.00,0.000000,0.011905,0.035714,0.00000,0.000000,0.035714
2,Central Harlem,0.000000,0.00,0.00,0.068182,0.045455,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
3,Chelsea,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.00000,0.010000,0.000000
4,Chinatown,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,...,0.00,0.030000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.010000
5,Civic Center,0.000000,0.00,0.00,0.000000,0.040000,0.01,0.000000,0.000000,0.000000,...,0.00,0.010000,0.000000,0.00,0.010000,0.010000,0.010000,0.01000,0.000000,0.030000
6,Clinton,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.040000,0.00000,0.000000,0.000000
7,East Harlem,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
8,East Village,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.000000,0.010000,0.010000,...,0.00,0.020000,0.000000,0.00,0.000000,0.030000,0.010000,0.00000,0.000000,0.000000
9,Financial District,0.000000,0.00,0.00,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.00000,0.010000,0.010000


In [369]:
jazz_grouped = manhattan_grouped[manhattan_grouped['Jazz Club']>0]

In [370]:
num_top_venues = 5

for hood in jazz_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = jazz_grouped[jazz_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1        Cosmetics Shop  0.05
2     French Restaurant  0.05
3    Seafood Restaurant  0.05
4  Gym / Fitness Center  0.05


----East Village----
                venue  freq
0  Mexican Restaurant  0.05
1                 Bar  0.05
2         Coffee Shop  0.04
3        Cocktail Bar  0.04
4           Speakeasy  0.03


----Greenwich Village----
                venue  freq
0  Italian Restaurant  0.10
1                Café  0.05
2    Sushi Restaurant  0.05
3                 Gym  0.03
4      Clothing Store  0.02


----Upper East Side----
                  venue  freq
0    Italian Restaurant  0.07
1           Coffee Shop  0.06
2  Gym / Fitness Center  0.05
3                Bakery  0.05
4           Yoga Studio  0.03


----West Village----
                     venue  freq
0       Italian Restaurant  0.07
1                 Wine Bar  0.05
2              Pizza Place  0.04
3  New American Restaurant  0.04
4             Co

In [371]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [379]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Memorial Site,Coffee Shop,Food Court,Beer Garden,Playground,Plaza,Shopping Mall
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Pizza Place,Gym / Fitness Center,Japanese Restaurant,Italian Restaurant,Gym,Wine Shop,Bookstore
2,Central Harlem,African Restaurant,Bar,Seafood Restaurant,American Restaurant,Gym / Fitness Center,French Restaurant,Chinese Restaurant,Cosmetics Shop,Caribbean Restaurant,Dessert Shop
3,Chelsea,Art Gallery,Coffee Shop,Ice Cream Shop,Café,Bakery,American Restaurant,Cocktail Bar,Italian Restaurant,Cycle Studio,Nightclub
4,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Coffee Shop,Spa,Optical Shop,Bar,Ice Cream Shop,Bubble Tea Shop,Vietnamese Restaurant


In [380]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [381]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 0, 0, 0, 1, 1, 3, 0, 1], dtype=int32)

In [385]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

which cluster label(s) do the neighborhoods with jazz clubs have?

In [395]:
manhattan_merged[manhattan_merged['Neighborhood'].isin(['Central Harlem','Upper East Side','Greenwich Village','East Village','West Village'])]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
105,Manhattan,Central Harlem,40.815976,-73.943211,0,African Restaurant,Bar,Seafood Restaurant,American Restaurant,Gym / Fitness Center,French Restaurant,Chinese Restaurant,Cosmetics Shop,Caribbean Restaurant,Dessert Shop
107,Manhattan,Upper East Side,40.775639,-73.960508,0,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center,French Restaurant,Yoga Studio,Juice Bar,Spa,Hotel,Mexican Restaurant
117,Manhattan,Greenwich Village,40.726933,-73.999914,0,Italian Restaurant,Sushi Restaurant,Café,Gym,Chinese Restaurant,Burger Joint,Pizza Place,Coffee Shop,Indie Movie Theater,Ice Cream Shop
118,Manhattan,East Village,40.727847,-73.982226,0,Bar,Mexican Restaurant,Cocktail Bar,Coffee Shop,Speakeasy,Wine Bar,Juice Bar,Ice Cream Shop,Pizza Place,Ramen Restaurant
123,Manhattan,West Village,40.734434,-74.006180,0,Italian Restaurant,Wine Bar,New American Restaurant,American Restaurant,Bakery,Cocktail Bar,Park,Pizza Place,Jazz Club,Coffee Shop


OK, cluster label 0 is unanimously the cluster for a Jazz Bar.

Finally, let's map the clustered neighborhoods along with markers for the existing Jazz clubs.

In [396]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
    
for idx, row in nearby_venues[nearby_venues['city']=='New York'].iterrows():
    Marker([row['lat'], row['lng']]).add_to(map_clusters)    
       
# Show the map
embed_map(map_clusters, 'map_clusters.html')

Jazz bars are linked with Cluster 0 most of the time.  Wine bars, speakeasys, Italian, French eating and a general cospopolitan feel about the neighborhood.  However, they are generally on the outskirts.

Question would be: safety in numbers and go where the other jazz clubs are, sharing the similar deomgraphic customers, or go for Sutton Place, or Lennox Hill which are cluster 0 without an existing Jazz club.